# Prevedere il crime rate

## Caricamento Librerie
Per prima cosa carichiamo le librerie per effettuare operazioni sui dati

*   NumPy per creare e operare su array a N dimensioni
*   pandas per caricare e manipolare dati tabulari
*   matplotlib per creare grafici

Importiamo le librerie usando i loro alias convenzionali

In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

## Caricamento dei dati
I dataset presi sono satti scricati dal sito ufficile delle nazioni unite https://dataunodc.un.org/ , e contengono varie informazioni tra cui:
*  Kidnapping
*  Rape
*  Drug Trafficking
*  Sexual assault
*  Burglary and Theft (insieme)
*  Homicide
I dataset presentano tutti dati dal 2013 al 2022 delle varie nazioni del mondo

### Significato delle colonne
In tutti i dataset che prendiamo in considerazione abbiamo le varie colonne:
*  Nation
*

## Crimini per furto

In [166]:
import os.path
file = "data_cts_corruption_and_economic_crime.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  theftCrime = pd.read_csv(file, index_col=False, encoding='latin1')

In [167]:
theftCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22754 entries, 0 to 22753
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Iso3_code            22754 non-null  object 
 1   Country              22754 non-null  object 
 2   Region               22754 non-null  object 
 3   Subregion            22754 non-null  object 
 4   Indicator            22754 non-null  object 
 5   Dimension            22754 non-null  object 
 6   Category             22754 non-null  object 
 7   Sex                  22754 non-null  object 
 8   Age                  22754 non-null  object 
 9   Year                 22754 non-null  int64  
 10  Unit of measurement  22754 non-null  object 
 11  VALUE                22754 non-null  float64
 12  Source               22754 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 16.5 MB


In [168]:
theftCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,ARM,Armenia,Asia,Western Asia,Offences,by type of offence,Corruption,Total,Total,2013,Counts,782.0,CTS
1,AUT,Austria,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,3439.0,CTS
2,CHE,Switzerland,Europe,Western Europe,Offences,by type of offence,Corruption,Total,Total,2013,Counts,4884.0,CTS
3,CHL,Chile,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,339.0,CTS
4,COL,Colombia,Americas,Latin America and the Caribbean,Offences,by type of offence,Corruption,Total,Total,2013,Counts,23483.0,CTS


In [169]:
theftCrime.shape

(22754, 13)

### Preprocessing dei dati
Filtrimo il dataframe utilizzando come unità solo 'Counts' inevce che 'Rate per 100,000 population'. Effettuiamo quindi una prima scrematura dei dati.

Selezioniamo inoltre soltanto le categorie interessate, ovvero nel nostro caso i `Theft` e i `Burglary`.

In [170]:
theftCrime = theftCrime[theftCrime['Unit of measurement'] != "Rate per 100,000 population"]
theftCrime = theftCrime[theftCrime['Category'].isin(['Theft', 'Burglary'])]

In [171]:
theftCrime.shape

(3563, 13)

Analizzando il dataframe, le feature 'Age', 'Sex', 'Iso3_code', 'Dimension', 'Unit of measurement'e 'Source' non sono rilevanti per il nostro modello, perciò effetuiamo la loro rimozione.

In [172]:
theftCrime = theftCrime.drop(['Unit of measurement', 'Sex', 'Iso3_code', 'Age', 'Source', 'Dimension', 'Indicator'], axis=1)

In [173]:
theftCrime.head()

,Country,Region,Subregion,Category,Year,VALUE
2486,United Arab Emirates,Asia,Western Asia,Burglary,2003,1882.0
2487,Australia,Oceania,Australia and New Zealand,Burglary,2003,354020.0
2488,Azerbaijan,Asia,Western Asia,Burglary,2003,757.0
2489,Belgium,Europe,Western Europe,Burglary,2003,97007.0
2490,Bulgaria,Europe,Eastern Europe,Burglary,2003,28210.0


Notimao inoltre che, dalle informazioni ricevute nel paragrafo precedente, la maggior parte delle feature tranne `Year` e `Value` sono di tipo `Object`, e questo perchè paython converte automaticamente in tipo objet tuttin i dati che non sono interi. Dunque cambiamo i vari tipi delle feature.
Inoltre, rimuoviamo, se vi sono, le varie righe conteneti valori nulli.

In [174]:
theftCrime["Country"] = theftCrime["Country"].astype(pd.StringDtype())
theftCrime["Region"] = theftCrime["Region"].astype(pd.StringDtype())
theftCrime["Subregion"] = theftCrime["Subregion"].astype(pd.StringDtype())
theftCrime["Category"] = theftCrime["Category"].astype(pd.StringDtype())
theftCrime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3563 entries, 2486 to 6048
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    3563 non-null   string 
 1   Region     3563 non-null   string 
 2   Subregion  3563 non-null   string 
 3   Category   3563 non-null   string 
 4   Year       3563 non-null   int64  
 5   VALUE      3563 non-null   float64
dtypes: float64(1), int64(1), string(4)
memory usage: 194.9 KB


In [175]:
theftCrime.dropna()
theftCrime.shape

(3563, 6)

Notiamo che il nostro dataset non contiene valori nulli in quanto la shape non è cambiata.

Infine, dovendo analizzare due crimini differenti, separiamo le due casitiche e rinominamo la colonna `VALUE` in `theft_count` e `burglary`.

In [176]:
theft = theftCrime.copy()
theft = theft[theft['Category'].isin(['Theft'])]
print(theft.shape)
theft.head()

(1904, 6)


,Country,Region,Subregion,Category,Year,VALUE
4145,United Arab Emirates,Asia,Western Asia,Theft,2003,10802.0
4146,Armenia,Asia,Western Asia,Theft,2003,2831.0
4147,Australia,Oceania,Australia and New Zealand,Theft,2003,624036.0
4148,Azerbaijan,Asia,Western Asia,Theft,2003,1820.0
4149,Belgium,Europe,Western Europe,Theft,2003,225000.0


In [177]:
burglary = theftCrime.copy()
burglary = burglary[burglary['Category'].isin(['Burglary'])]
print(burglary.shape)
burglary.head()

(1659, 6)


,Country,Region,Subregion,Category,Year,VALUE
2486,United Arab Emirates,Asia,Western Asia,Burglary,2003,1882.0
2487,Australia,Oceania,Australia and New Zealand,Burglary,2003,354020.0
2488,Azerbaijan,Asia,Western Asia,Burglary,2003,757.0
2489,Belgium,Europe,Western Europe,Burglary,2003,97007.0
2490,Bulgaria,Europe,Eastern Europe,Burglary,2003,28210.0


In [178]:
theft.drop(['Category'], axis=1, inplace=True)
burglary.drop(['Category'], axis=1, inplace=True)

In [179]:
theft.rename(columns={'VALUE':'theft_count'}, inplace=True)
burglary.rename(columns={'VALUE':'burglary_count'}, inplace=True)

In [180]:
theft.describe()

,Year,theft_count
count,1904.000000,1.904000e+03
mean,2012.284139,2.041721e+05
std,5.507616,6.686822e+05
min,2003.000000,0.000000e+00
25%,2008.000000,5.014500e+03
50%,2012.000000,2.516400e+04
75%,2017.000000,1.247930e+05
max,2022.000000,7.026802e+06


In [181]:
burglary.describe()

,Year,burglary_count
count,1659.000000,1.659000e+03
mean,2012.474382,7.028198e+04
std,5.468942,2.145695e+05
min,2003.000000,0.000000e+00
25%,2008.000000,1.513000e+03
50%,2012.000000,9.703000e+03
75%,2017.000000,5.444000e+04
max,2022.000000,2.228887e+06


Contiamo anche il numero di paesi nei nostri due dataframe.

In [182]:
print(f"Numero totale di paesi in theft: {theft['Country'].unique().size}")
print(f"Numero totale di paesi in burglary: {burglary['Country'].unique().size}")

Numero totale di paesi in theft: 146
Numero totale di paesi in burglary: 138


## Crimini per omicidio

In [183]:
file = "data_cts_intentional_homicide.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  homicideCrime = pd.read_csv(file, index_col=False, encoding='latin1')

In [184]:
homicideCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117070 entries, 0 to 117069
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Iso3_code            117070 non-null  object 
 1   Country              117070 non-null  object 
 2   Region               117070 non-null  object 
 3   Subregion            117070 non-null  object 
 4   Indicator            117070 non-null  object 
 5   Dimension            117070 non-null  object 
 6   Category             117070 non-null  object 
 7   Sex                  117070 non-null  object 
 8   Age                  117070 non-null  object 
 9   Year                 117070 non-null  int64  
 10  Unit of measurement  117070 non-null  object 
 11  VALUE                117070 non-null  float64
 12  Source               117070 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 87.4 MB


In [185]:
homicideCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,ARM,Armenia,Asia,Western Asia,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,35.0,CTS
1,CHE,Switzerland,Europe,Western Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,28.0,CTS
2,COL,Colombia,Americas,Latin America and the Caribbean,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,15053.0,CTS
3,CZE,Czechia,Europe,Eastern Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,69.0,CTS
4,DEU,Germany,Europe,Western Europe,Persons arrested/suspected for intentional hom...,by citizenship,National citizens,Male,Total,2013,Counts,455.0,CTS


In [186]:
homicideCrime.shape

(117070, 13)

### Preprocessing dei dati
Filtrimo il dataframe utilizzando come unità solo 'Counts' inevce che 'Rate per 100,000 population'. Effettuiamo quindi una prima scrematura dei dati

In [187]:
homicideCrime = homicideCrime[homicideCrime['Unit of measurement'] != "Rate per 100,000 population"]
homicideCrime.shape

(62850, 13)

Poichè stiamo analizzando i crimini legati all'omicidio dobbiamo fare un'ulteriore scrematura in modo da considerare esclusivamente i crimini di nostro interesse. In particolare di seguito controllo quali categorie riguardano gli omicidi.

In [188]:
homicide_categories = homicideCrime[homicideCrime['Indicator'].str.contains('homicide', case=False, na=False)]['Indicator'].unique()
print(homicide_categories)

['Persons arrested/suspected for intentional homicide'
 'Victims of intentional homicide'
 'Victims of intentional homicide â\x80\x93 City-level data'
 'Persons convicted for intentional homicide'
 'Death due to intentional homicide in prison'
 'Victims of Intentional Homicide - Regional Estimate']


In [189]:
homicideCrime = homicideCrime[homicideCrime['Indicator'].isin(['Victims of intentional homicide'])]
homicideCrime.shape

(46224, 13)

Così facendo rimuoviamo, come precedentemente, buona parte dei dati e allegeriamo così il dataset.

Come prima, rimuoviamo anche da questo dataset le feature non rilevanti per lo studio. In questo caso modifichiamo la struttura del dataset per una miglire comprensione e coerenza, inserendo in `Category` il valore `Homicide` che era di `Indicator`.

Inoltre, effettuiamo anche la conversione delle feature da `Object` in `String`.

In [190]:
homicideCrime = homicideCrime.drop(['Unit of measurement', 'Sex', 'Iso3_code', 'Age', 'Source', 'Dimension', 'Category'], axis=1)
homicideCrime.rename(columns={'Indicator':'Category'}, inplace=True)
homicideCrime["Country"] = homicideCrime["Country"].astype(pd.StringDtype())
homicideCrime["Region"] = homicideCrime["Region"].astype(pd.StringDtype())
homicideCrime["Subregion"] = homicideCrime["Subregion"].astype(pd.StringDtype())
homicideCrime["Category"] = homicideCrime["Category"].astype(pd.StringDtype())

homicideCrime['Category'] = homicideCrime['Category'].replace('Victims of intentional homicide', 'Homicide')
homicideCrime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46224 entries, 1408 to 47631
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    46224 non-null  string 
 1   Region     46224 non-null  string 
 2   Subregion  46224 non-null  string 
 3   Category   46224 non-null  string 
 4   Year       46224 non-null  int64  
 5   VALUE      46224 non-null  float64
dtypes: float64(1), int64(1), string(4)
memory usage: 2.5 MB


In [191]:
homicideCrime.head()

,Country,Region,Subregion,Category,Year,VALUE
1408,Aruba,Americas,Latin America and the Caribbean,Homicide,1990,0.0
1409,Anguilla,Americas,Latin America and the Caribbean,Homicide,1990,0.0
1410,Armenia,Asia,Western Asia,Homicide,1990,178.0
1411,Antigua and Barbuda,Americas,Latin America and the Caribbean,Homicide,1990,1.0
1412,Australia,Oceania,Australia and New Zealand,Homicide,1990,374.0


In [192]:
homicideCrime.shape

(46224, 6)

In [193]:
homicideCrime.dropna()
homicideCrime.shape

(46224, 6)

Notimao anche questa volta che la shape del dataset non è cambiata dopo aver effettuato la `dropna()`, il che vuol dire non vi sono presenti righr con valori nulli.





In [194]:
homicideCrime.rename(columns={'VALUE':'homicide_count'}, inplace=True)
homicideCrime.describe()

,Year,homicide_count
count,46224.000000,46224.000000
mean,2014.777021,510.106158
std,6.462734,3091.851061
min,1990.000000,0.000000
25%,2012.000000,1.000000
50%,2016.000000,13.000000
75%,2019.000000,76.000000
max,2023.000000,63788.000000


Controlliamo il numero di paesi del dataset:

In [195]:
print(f"Numero totale di paesi: {homicideCrime['Country'].unique().size}")

Numero totale di paesi: 205


In questo caso, il numero di peasi è lo stesso del dataset precedente, ma non contiene gli stessi paesi.

Dunque, per poter procedere verso il merging dei vari dataset per creare uno solo, bisogna rimuovere da `theftCrime` tutte le righe rigurdanti i paesi non contenuti in `homicideCrime`.

Controlliamo intanto i paesi che differenziano da un dataset all'altro.

In [196]:
different_in_theft = countries_theft - countries_homicide
different_in_homicide = countries_homicide - countries_theft

if different_in_theft:
    print(f"Paesi in theftCrime ma non in homicideCrime: {different_in_theft}")
    print(f"Numero: {len(different_in_theft)}")
if different_in_homicide:
    print(f"Paesi in homicideCrime ma non in theftCrime: {different_in_homicide}")
    print(f"Numero: {len(different_in_homicide)}")

Paesi in theftCrime ma non in homicideCrime: {'Djibouti', 'Sudan', 'Guinea', 'Senegal', 'Kyrgyzstan', 'CÃ´te dâ\x80\x99Ivoire'}
Numero: 6
Paesi in homicideCrime ma non in theftCrime: {'Eritrea', 'Tonga', 'Cook Islands', 'Uzbekistan', 'Iraq', 'South Africa', 'Samoa', 'CuraÃ§ao', 'Gibraltar', 'Malawi', 'Cambodia', 'Ethiopia', 'Cayman Islands', 'French Polynesia', 'Liberia', 'Papua New Guinea', 'Montserrat', 'Turks and Caicos Islands', 'Ghana', 'Martinique', 'Cuba', 'Tunisia', 'Vanuatu', 'Palau', 'Seychelles', 'Iraq (Kurdistan Region)', 'Guadeloupe', 'Anguilla', 'Greenland', 'Viet Nam', 'Saudi Arabia', 'San Marino', 'French Guiana', 'Micronesia (Federated States of)', 'Mauritania', 'China', 'Zambia', 'Haiti', 'Namibia', 'Saint Martin (French Part)', 'South Sudan', 'Guam', 'Afghanistan', 'Kiribati', 'Angola', 'Saint Helena', 'Venezuela (Bolivarian Republic of)', 'Fiji', 'Isle of Man', 'American Samoa', 'New Caledonia', 'Saint Pierre and Miquelon', 'British Virgin Islands', 'Marshall Island

In [197]:
common_countries = countries_theft & countries_homicide
print(f"Numero Paesi in comune: {len(common_countries)}")

Numero Paesi in comune: 144


Il numero di paesi in comune è 143, quindi muteremo i due dataset filtrandoli in base ai paesi in comune tra i due.

In [198]:
theftCrime = theftCrime[theftCrime['Country'].isin(common_countries)]
homicideCrime = homicideCrime[homicideCrime['Country'].isin(common_countries)]
print(f"theftCrime: {theftCrime.shape}")
print(f"homicideCrime: {homicideCrime.shape}")

theftCrime: (3520, 6)
homicideCrime: (44759, 6)


## Violenza sessuale e rapimenti

In [199]:
file = "data_cts_violent_and_sexual_crime.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  violent_sexualCrime = pd.read_csv(file, index_col=False, encoding='latin1')

In [200]:
violent_sexualCrime.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26114 entries, 0 to 26113
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Iso3_code            26114 non-null  object 
 1   Country              26114 non-null  object 
 2   Region               26114 non-null  object 
 3   Subregion            26114 non-null  object 
 4   Indicator            26114 non-null  object 
 5   Dimension            26114 non-null  object 
 6   Category             26114 non-null  object 
 7   Sex                  26114 non-null  object 
 8   Age                  26114 non-null  object 
 9   Year                 26114 non-null  int64  
 10  Unit of measurement  26114 non-null  object 
 11  VALUE                26114 non-null  float64
 12  Source               26114 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 19.3 MB


In [201]:
violent_sexualCrime.shape

(26114, 13)

In [202]:
violent_sexualCrime.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,AZE,Azerbaijan,Asia,Western Asia,Violent offences,by type of offence,Serious assault,Total,Total,2003,Counts,155.0,CTS
1,BEL,Belgium,Europe,Western Europe,Violent offences,by type of offence,Serious assault,Total,Total,2003,Counts,61959.0,CTS
2,BGR,Bulgaria,Europe,Eastern Europe,Violent offences,by type of offence,Serious assault,Total,Total,2003,Counts,3806.0,CTS
3,BHR,Bahrain,Asia,Western Asia,Violent offences,by type of offence,Serious assault,Total,Total,2003,Counts,2701.0,CTS
4,BLR,Belarus,Europe,Eastern Europe,Violent offences,by type of offence,Serious assault,Total,Total,2003,Counts,4032.0,CTS


Filtrimo il dataframe, manteniamo solo i dati che presentano come `Unit of measurement` `Counts` inevce che `Rate per 100,000 population`.

In [203]:
violent_sexualCrime = violent_sexualCrime[violent_sexualCrime['Unit of measurement'] != "Rate per 100,000 population"]
violent_sexualCrime.shape

(13073, 13)

Le feature `Iso3_code`, `Indicator`, `Dimension`, `Sex`, `Age`, `Unit of measurement` e `Source` sono inutili per il nostro modello, perciò le rimuoviamo.
Inoltre rimuoviamo le eventuali righe contenenti valori null.

In [204]:
violent_sexualCrime = violent_sexualCrime.drop(['Iso3_code', 'Indicator', 'Dimension', 'Sex', 'Age', 'Unit of measurement', 'Source'], axis=1)
violent_sexualCrime.dropna()

,Country,Region,Subregion,Category,Year,VALUE
0,Azerbaijan,Asia,Western Asia,Serious assault,2003,155.0
1,Belgium,Europe,Western Europe,Serious assault,2003,61959.0
2,Bulgaria,Europe,Eastern Europe,Serious assault,2003,3806.0
3,Bahrain,Asia,Western Asia,Serious assault,2003,2701.0
4,Belarus,Europe,Eastern Europe,Serious assault,2003,4032.0
...,...,...,...,...,...,...
13068,Montenegro,Europe,Southern Europe,Acts intended to induce fear or emotional dist...,2021,10.0
13069,Mauritius,Africa,Sub-Saharan Africa,Acts intended to induce fear or emotional dist...,2021,342.0
13070,El Salvador,Americas,Latin America and the Caribbean,Acts intended to induce fear or emotional dist...,2021,4.0
13071,Serbia,Europe,Southern Europe,Acts intended to induce fear or emotional dist...,2021,1.0


Notiamo che il numero di righe del dataset non è cambiata dopo aver effettuato la dropna(), il che vuol dire non vi sono presenti righe con valori nulli.

Di seguito, invece, effettuiamo la conversione da Object a String delle feature che lo necessitano.

In [205]:
violent_sexualCrime["Country"] = violent_sexualCrime["Country"].astype(pd.StringDtype())
violent_sexualCrime["Region"] = violent_sexualCrime["Region"].astype(pd.StringDtype())
violent_sexualCrime["Subregion"] = violent_sexualCrime["Subregion"].astype(pd.StringDtype())
violent_sexualCrime["Category"] = violent_sexualCrime["Category"].astype(pd.StringDtype())
violent_sexualCrime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13073 entries, 0 to 13072
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    13073 non-null  string 
 1   Region     13073 non-null  string 
 2   Subregion  13073 non-null  string 
 3   Category   13073 non-null  string 
 4   Year       13073 non-null  int64  
 5   VALUE      13073 non-null  float64
dtypes: float64(1), int64(1), string(4)
memory usage: 714.9 KB


Poichè dobbiamo analizzare i crimini di violenza sessuale ed i rapimenti andiamo a creare due copie del dataframe in modo da poter analizzare separatamente i due casi.

Inoltre rinominiamo la feature `VALUE` in `sexual_violence_count` e `kidnapping_count` a seconda del fenomeno analizzato.

In [206]:
sexual_violence = violent_sexualCrime.copy()
sexual_violence = sexual_violence[sexual_violence['Category'].isin(['Sexual violence'])]
sexual_violence.head()

,Country,Region,Subregion,Category,Year,VALUE
3796,Belgium,Europe,Western Europe,Sexual violence,2003,5565.0
3797,Bulgaria,Europe,Eastern Europe,Sexual violence,2003,1287.0
3798,Canada,Americas,Northern America,Sexual violence,2003,26128.0
3799,Czechia,Europe,Eastern Europe,Sexual violence,2003,1898.0
3800,Germany,Europe,Western Europe,Sexual violence,2003,54632.0


In [207]:
kidnapping = violent_sexualCrime.copy()
kidnapping = kidnapping[kidnapping['Category'].isin(['Kidnapping'])]
kidnapping.head()

,Country,Region,Subregion,Category,Year,VALUE
1977,United Arab Emirates,Asia,Western Asia,Kidnapping,2003,434.0
1978,Azerbaijan,Asia,Western Asia,Kidnapping,2003,28.0
1979,Belgium,Europe,Western Europe,Kidnapping,2003,1003.0
1980,Bulgaria,Europe,Eastern Europe,Kidnapping,2003,209.0
1981,Bahrain,Asia,Western Asia,Kidnapping,2003,5.0


Avendo effetuato questa ulteriore scrematura posso eliminare anche la colonna `Category`.

In [208]:
sexual_violence = sexual_violence.drop(['Category'], axis=1)
sexual_violence.rename(columns={'VALUE':'sexual_violence_count'}, inplace=True)
sexual_violence.shape

(1768, 5)

In [209]:
sexual_violence.describe()

,Year,sexual_violence_count
count,1768.000000,1768.000000
mean,2013.143100,6434.749434
std,5.244384,15363.198967
min,2003.000000,0.000000
25%,2009.000000,238.000000
50%,2013.000000,1362.500000
75%,2018.000000,5163.250000
max,2022.000000,193566.000000


In [210]:
kidnapping = kidnapping.drop(['Category'], axis=1)
kidnapping.rename(columns={'VALUE':'kidnapping_count'}, inplace=True)
kidnapping.shape

(1819, 5)

In [211]:
kidnapping.describe()

,Year,kidnapping_count
count,1819.000000,1819.000000
mean,2012.722375,792.048679
std,5.438161,4022.973880
min,2003.000000,0.000000
25%,2008.000000,5.000000
50%,2013.000000,34.000000
75%,2017.000000,265.000000
max,2022.000000,65461.000000


Notiamo che il dataset, per ogni `Country` contiene dati dal 2003 al 2022.

In [212]:
print(f"Numero totale di paesi nel dataframe sexual_violence: {sexual_violence['Country'].unique().size}")

Numero totale di paesi nel dataframe sexual_violence: 140


In [213]:
print(f"Numero totale di paesi nel dataframe kidnapping: {kidnapping['Country'].unique().size}")

Numero totale di paesi nel dataframe kidnapping: 147


## Stipendio medio

In [253]:
file = "employment.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  employment = pd.read_csv(file, index_col=False, encoding='latin1')

In [254]:
employment.head()

,"ï»¿""Country Name""",Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,65.222674,65.063834,64.969923,64.845470,64.649308,63.560774,63.612563,64.302928,64.436966,NaN
2,Afghanistan,AFG,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,42.924000,42.472000,42.021000,40.493000,38.965000,36.711000,36.263000,32.333000,31.268000,NaN
3,Africa Western and Central,AFW,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,59.763263,59.755348,59.372463,59.259533,59.158366,58.261096,58.490208,59.433005,59.620615,NaN
4,Angola,AGO,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,63.656000,63.317000,63.028000,62.793000,62.642000,62.550000,64.304000,64.447000,64.492000,NaN


Modifichiamo il dataset sostituendo i valori nulli NaN con zeri. Successivamente faremo una scrematura per eliminare dal nostro dataset tutti gli anni con valore zero, che prima era NaN.

In [255]:
employment.dropna()
employment.fillna(0)
employment.shape


(266, 69)

In [256]:
employment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 69 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ï»¿"Country Name"  266 non-null    object 
 1   Country Code       266 non-null    object 
 2   Indicator Name     266 non-null    object 
 3   Indicator Code     266 non-null    object 
 4   1960               0 non-null      float64
 5   1961               0 non-null      float64
 6   1962               0 non-null      float64
 7   1963               0 non-null      float64
 8   1964               0 non-null      float64
 9   1965               0 non-null      float64
 10  1966               0 non-null      float64
 11  1967               0 non-null      float64
 12  1968               0 non-null      float64
 13  1969               0 non-null      float64
 14  1970               0 non-null      float64
 15  1971               0 non-null      float64
 16  1972               0 non-n

Compo si può osservare, non tutte le colonne degli anni presentano dei dati, quindi facciamo una prima scrematura togliendo le colonne che non sono rilevanti e rinominandole dove necessario.

In [257]:
columns = employment.columns[:]
columns

Index(['ï»¿"Country Name"', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Unnamed: 68'],
      dtype='object')

In [258]:
employment.rename(columns={'ï»¿"Country Name"':'Country'}, inplace=True)

In [259]:
columns_to_drop = [str(year) for year in range(1960, 1991)]
columns_to_drop = [col for col in columns_to_drop if col in employment.columns]
employment.drop(columns=columns_to_drop, inplace=True)

In [260]:
employment.drop(columns=['Unnamed: 68'], inplace=True)

In [261]:
employment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1991            235 non-null    float64
 5   1992            235 non-null    float64
 6   1993            235 non-null    float64
 7   1994            235 non-null    float64
 8   1995            235 non-null    float64
 9   1996            235 non-null    float64
 10  1997            235 non-null    float64
 11  1998            235 non-null    float64
 12  1999            235 non-null    float64
 13  2000            235 non-null    float64
 14  2001            235 non-null    float64
 15  2002            235 non-null    float64
 16  2003            235 non-null    float64
 17  2004            235 non-null    flo

A questo punto, per una maggiore coerenza con i dati dei dataet precedenti, e una migliore organizzazione, convertiamo le colonne dei singoli anni in una unica colonna `Year`, dove verranno ripetute tante istanze di ogni paese, quante sono i dati rilevanti negli anni.

In [262]:
employment_melted = employment.copy()

employment_melted = pd.melt(employment_melted,
                            id_vars=['Country', 'Country Code', 'Indicator Name', 'Indicator Code'],
                            var_name='Year', value_name='Annual_Employment_Perc')

employment_melted['Year'] = pd.to_numeric(employment_melted['Year'], errors='coerce')

employment_filtered = employment_melted.dropna(subset=['Annual_Employment_Perc']).copy()

employment_filtered.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)

employment_filtered.head()

,Country,Country Code,Year,Annual_Employment_Perc
1,Africa Eastern and Southern,AFE,1991,66.041530
2,Afghanistan,AFG,1991,43.256000
3,Africa Western and Central,AFW,1991,63.236562
4,Angola,AGO,1991,64.525000
5,Albania,ALB,1991,57.568000


Completaimo la preparazione del dataset con la conversione dei tipi delle feature.

In [263]:
employment_filtered["Country"] = employment_filtered["Country"].astype(pd.StringDtype())
employment_filtered["Country Code"] = employment_filtered["Country Code"].astype(pd.StringDtype())
employment_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7752 entries, 1 to 8777
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 7752 non-null   string 
 1   Country Code            7752 non-null   string 
 2   Year                    7752 non-null   int64  
 3   Annual_Employment_Perc  7752 non-null   float64
dtypes: float64(1), int64(1), string(2)
memory usage: 302.8 KB


In [264]:
employment_filtered.describe()

,Year,Annual_Employment_Perc
count,7752.000000,7752.000000
mean,2006.993937,56.495562
std,9.519366,10.794473
min,1991.000000,22.634000
25%,1999.000000,50.521439
50%,2007.000000,56.821429
75%,2015.000000,62.912287
max,2023.000000,88.753000


## Inflazione nei vari paesi

In [265]:
file = "annual_infation.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  inflation = pd.read_csv(file, index_col=False, encoding='latin1')

In [266]:
print(inflation.shape)
inflation.head()

(266, 69)


,"ï»¿""Country Name""",Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.245878,6.571396,6.399343,4.720805,4.653665,7.321106,6.824727,10.883478,7.399186,NaN
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.130817,1.487416,1.725486,1.784050,1.760112,2.437609,3.653533,7.967574,4.670084,NaN
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,9.355972,30.694415,29.844480,19.628938,17.080954,22.271539,25.754295,21.355290,13.644102,NaN


In [267]:
inflation.fillna(0)
inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 69 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ï»¿"Country Name"  266 non-null    object 
 1   Country Code       266 non-null    object 
 2   Indicator Name     266 non-null    object 
 3   Indicator Code     266 non-null    object 
 4   1960               70 non-null     float64
 5   1961               72 non-null     float64
 6   1962               74 non-null     float64
 7   1963               74 non-null     float64
 8   1964               79 non-null     float64
 9   1965               86 non-null     float64
 10  1966               93 non-null     float64
 11  1967               99 non-null     float64
 12  1968               100 non-null    float64
 13  1969               101 non-null    float64
 14  1970               106 non-null    float64
 15  1971               110 non-null    float64
 16  1972               113 non

Anche in questo caso effettuiamo una prima filtazione dei dati, eliminando le colonne non ritenute necessarie, e rinominandone altre.

In [268]:
columns = inflation.columns[:]
columns

Index(['ï»¿"Country Name"', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Unnamed: 68'],
      dtype='object')

In [269]:
inflation.rename(columns={'ï»¿"Country Name"':'Country'}, inplace=True)
inflation.drop(columns=['Unnamed: 68'], inplace=True)
inflation.drop(columns=columns_to_drop, inplace=True)

In [270]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1991            179 non-null    float64
 5   1992            187 non-null    float64
 6   1993            193 non-null    float64
 7   1994            198 non-null    float64
 8   1995            202 non-null    float64
 9   1996            205 non-null    float64
 10  1997            205 non-null    float64
 11  1998            205 non-null    float64
 12  1999            207 non-null    float64
 13  2000            211 non-null    float64
 14  2001            216 non-null    float64
 15  2002            218 non-null    float64
 16  2003            221 non-null    float64
 17  2004            222 non-null    flo

Anche in questo caso modifichiamo e rimodelliamo il dataset in modo che gli anni vengano tutti raccolti nella colonna `Year`.

In [271]:
inflation_melted = inflation.copy()

inflation_melted = pd.melt(inflation_melted,
                            id_vars=['Country', 'Country Code', 'Indicator Name', 'Indicator Code'],
                            var_name='Year', value_name='Annual_Inflation_Perc')

inflation_melted['Year'] = pd.to_numeric(inflation_melted['Year'], errors='coerce')

inflation_filtered = inflation_melted.dropna(subset=['Annual_Inflation_Perc']).copy()

inflation_filtered.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)

inflation_filtered.head()

,Country,Country Code,Year,Annual_Inflation_Perc
0,Aruba,ABW,1991,5.555556
1,Africa Eastern and Southern,AFE,1991,17.678100
3,Africa Western and Central,AFW,1991,1.741888
4,Angola,AGO,1991,83.783784
7,Arab World,ARB,1991,9.000000


In [272]:
inflation_filtered["Country"] = inflation_filtered["Country"].astype(pd.StringDtype())
inflation_filtered["Country Code"] = inflation_filtered["Country Code"].astype(pd.StringDtype())
inflation_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7203 entries, 0 to 8776
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                7203 non-null   string 
 1   Country Code           7203 non-null   string 
 2   Year                   7203 non-null   int64  
 3   Annual_Inflation_Perc  7203 non-null   float64
dtypes: float64(1), int64(1), string(2)
memory usage: 281.4 KB


In [273]:
inflation_filtered.describe()

,Year,Annual_Inflation_Perc
count,7203.000000,7203.000000
mean,2007.435096,19.153457
std,9.287850,309.600432
min,1991.000000,-16.859691
25%,2000.000000,2.083968
50%,2008.000000,4.120816
75%,2015.000000,7.890800
max,2023.000000,23773.131774


## Personale di polizia

In [39]:
file = "data_cts_access_and_functioning_of_justice.csv"
if not os.path.exists(file):
    print("Missing dataset")
else:
  police_personnel = pd.read_csv(file, index_col=False, encoding='latin1')

In [40]:
police_personnel.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102032 entries, 0 to 102031
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Iso3_code            102032 non-null  object 
 1   Country              102032 non-null  object 
 2   Region               102032 non-null  object 
 3   Subregion            102032 non-null  object 
 4   Indicator            102032 non-null  object 
 5   Dimension            102032 non-null  object 
 6   Category             102032 non-null  object 
 7   Sex                  102032 non-null  object 
 8   Age                  102032 non-null  object 
 9   Year                 102032 non-null  int64  
 10  Unit of measurement  102032 non-null  object 
 11  VALUE                102032 non-null  float64
 12  Source               102032 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 74.7 MB


In [41]:
police_personnel.shape

(102032, 13)

In [42]:
police_personnel.head()

,Iso3_code,Country,Region,Subregion,Indicator,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE,Source
0,AZE,Azerbaijan,Asia,Western Asia,Persons arrested/cautioned/suspected,by selected crime,Intentional Homicide,Total,Total,2003,Counts,175.0,CTS
1,BGR,Bulgaria,Europe,Eastern Europe,Persons arrested/cautioned/suspected,by selected crime,Intentional Homicide,Total,Total,2003,Counts,227.0,CTS
2,BHR,Bahrain,Asia,Western Asia,Persons arrested/cautioned/suspected,by selected crime,Intentional Homicide,Total,Total,2003,Counts,3.0,CTS
3,BLR,Belarus,Europe,Eastern Europe,Persons arrested/cautioned/suspected,by selected crime,Intentional Homicide,Total,Total,2003,Counts,944.0,CTS
4,BMU,Bermuda,Americas,Northern America,Persons arrested/cautioned/suspected,by selected crime,Intentional Homicide,Total,Total,2003,Counts,12.0,CTS


In [43]:
police_personnel = police_personnel[police_personnel['Unit of measurement'] != "Rate per 100,000 population"]
police_personnel.shape

(61270, 13)

In [44]:
police_personnel = police_personnel.drop(['Indicator', 'Dimension', 'Sex', 'Age', 'Unit of measurement', 'Source'], axis=1)
police_personnel.dropna()

,Iso3_code,Country,Region,Subregion,Category,Year,VALUE
0,AZE,Azerbaijan,Asia,Western Asia,Intentional Homicide,2003,175.0
1,BGR,Bulgaria,Europe,Eastern Europe,Intentional Homicide,2003,227.0
2,BHR,Bahrain,Asia,Western Asia,Intentional Homicide,2003,3.0
3,BLR,Belarus,Europe,Eastern Europe,Intentional Homicide,2003,944.0
4,BMU,Bermuda,Americas,Northern America,Intentional Homicide,2003,12.0
...,...,...,...,...,...,...,...
61265,SVK,Slovakia,Europe,Eastern Europe,Prison Staff - responsible for administration/...,2021,1747.0
61266,SVN,Slovenia,Europe,Southern Europe,Prison Staff - responsible for administration/...,2021,173.0
61267,SWE,Sweden,Europe,Northern Europe,Prison Staff - responsible for administration/...,2021,1160.0
61268,TUR,TÃ¼rkiye,Asia,Western Asia,Prison Staff - responsible for administration/...,2021,9446.0


Notiamo che il numero di righe del dataset non è cambiata dopo aver effettuato la dropna(), il che vuol dire non vi sono presenti righe con valori nulli.

Di seguito, invece, effettuiamo la conversione da Object a String delle feature che lo necessitano.

In [45]:
police_personnel["Iso3_code"] = police_personnel["Iso3_code"].astype(pd.StringDtype())
police_personnel["Country"] = police_personnel["Country"].astype(pd.StringDtype())
police_personnel["Region"] = police_personnel["Region"].astype(pd.StringDtype())
police_personnel["Subregion"] = police_personnel["Subregion"].astype(pd.StringDtype())
police_personnel["Category"] = police_personnel["Category"].astype(pd.StringDtype())
police_personnel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61270 entries, 0 to 61269
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Iso3_code  61270 non-null  string 
 1   Country    61270 non-null  string 
 2   Region     61270 non-null  string 
 3   Subregion  61270 non-null  string 
 4   Category   61270 non-null  string 
 5   Year       61270 non-null  int64  
 6   VALUE      61270 non-null  float64
dtypes: float64(1), int64(1), string(5)
memory usage: 3.7 MB


In [46]:
police_personnel = police_personnel[police_personnel['Category'].isin(['Police personel'])]
police_personnel.head()

,Iso3_code,Country,Region,Subregion,Category,Year,VALUE
49116,ALB,Albania,Europe,Southern Europe,Police personel,2003,12454.0
49117,AUS,Australia,Oceania,Australia and New Zealand,Police personel,2003,44185.0
49118,BEL,Belgium,Europe,Western Europe,Police personel,2003,36318.0
49119,BHR,Bahrain,Asia,Western Asia,Police personel,2003,13634.0
49120,BLR,Belarus,Europe,Eastern Europe,Police personel,2003,34946.0


In [51]:
police_personnel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510 entries, 49116 to 52625
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Iso3_code               3510 non-null   string 
 1   Country                 3510 non-null   string 
 2   Region                  3510 non-null   string 
 3   Subregion               3510 non-null   string 
 4   Year                    3510 non-null   int64  
 5   police_personnel_count  3510 non-null   float64
dtypes: float64(1), int64(1), string(4)
memory usage: 192.0 KB


Avendo effetuato questa ulteriore scrematura posso eliminare anche la colonna Category. 

Inoltre rinominiamo la feature VALUE in police_personnel_count.

In [48]:
police_personnel = police_personnel.drop(['Category'], axis=1)
police_personnel.rename(columns={'VALUE':'police_personnel_count'}, inplace=True)
police_personnel.shape

(3510, 6)

In [49]:
police_personnel.describe()

,Year,police_personnel_count
count,3510.000000,3.510000e+03
mean,2012.548148,5.771902e+04
std,4.813103,1.547414e+05
min,2003.000000,0.000000e+00
25%,2009.000000,2.678250e+03
50%,2012.000000,9.869500e+03
75%,2016.000000,3.884000e+04
max,2021.000000,1.731537e+06


Notiamo che il dataset, per ogni Country contiene dati dal 2003 al 2021.

In [50]:
print(f"Numero totale di paesi nel dataframe police_personnel: {police_personnel['Country'].unique().size}")

Numero totale di paesi nel dataframe police_personnel: 132
